In [1]:
#import all library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import library for VIF
#from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
# read csv file
housing = pd.read_csv('../data/cleaned_combined_data_wo_lat_long.csv')
housing.head()
housing.shape

(166542, 60)

### Feature_Engineering

In [3]:
# creating another feature - diff_from_max_lvl
#housing['diff_from_max_lvl'] = housing['max_floor_lvl'] - housing['mid_storey']

# creating another feature - percent_rental_flat
housing['percent_rental_flat'] = (housing['1room_rental'] + housing['2room_rental'] + housing['3room_rental'] + housing['other_room_rental'])/housing['total_dwelling_units']


# creating another feature - percent_5_room_and_above
housing['percent_4room_and_above_flat'] = (housing['4room_sold'] + housing['5room_sold'] + housing['exec_sold'] + housing['multigen_sold'] + housing['studio_apartment_sold'])/housing['total_dwelling_units']


rooms = ['3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE','MUTI_GENERATION']

# creating another feature - If the resale unit is 3 room-flat and above, and there are 1 room-flat or 2 room-flat, then 1 else 0
housing['present_of_3room_and_below'] = (
    (housing['flat_type'].isin(rooms)) & 
    ((housing['1room_sold'] + housing['2room_sold'] + housing['3room_sold']) > 0)
).astype(int)

# Interaction Term 
housing['distance_for_hawker_and_mrt'] = housing['hawker_nearest_distance'] * housing['mrt_nearest_distance']
housing['distance_for_mall_and_mrt'] = housing['mall_nearest_distance'] * housing['mrt_nearest_distance']



Dropping columns that is not going to be used in the regression model 

In [4]:
cols_to_drop = ['tranc_yearmonth']
cols_for_corr_test = housing.drop(columns = cols_to_drop , inplace = False)


cols_for_corr_test.corr()['resale_price'].sort_values()

3room_sold                     -0.411948
present_of_3room_and_below     -0.396862
hdb_age                        -0.348430
2room_sold                     -0.146455
total_dwelling_units           -0.144782
commercial                     -0.139708
mrt_nearest_distance           -0.127939
distance_for_mall_and_mrt      -0.126376
percent_rental_flat            -0.104979
mall_nearest_distance          -0.087096
3room_rental                   -0.080059
2room_rental                   -0.077612
distance_for_hawker_and_mrt    -0.047101
1room_sold                     -0.039488
cutoff_point_rank              -0.038822
1room_rental                   -0.037992
hawker_food_stalls             -0.032899
tranc_year                     -0.024679
vacancy                        -0.017042
hawker_nearest_distance        -0.016078
pri_sch_nearest_distance       -0.010871
hawker_market_stalls           -0.008555
multistorey_carpark            -0.007767
market_hawker                  -0.005925
tranc_month     

In [5]:
cols_for_corr_test['percent_4room_and_above_flat'].corr(cols_for_corr_test['present_of_3room_and_below'])

-0.862732413867657

### Drop Columns

Dropping columns that might have high multicollinearity and variables used to create interaction term 
- region, storey_range, full_flat_type and etc
- mall_nearest_distance, mrt_nearest_distance,hawker_nearest_distance

In [6]:
cols_to_drop = ['full_flat_type','present_of_3room_and_below','bus_stop_name','pri_sch_name','tranc_year','tranc_month','mall_nearest_distance','mrt_nearest_distance','hawker_nearest_distance','region','storey_range','1room_sold','2room_sold','3room_sold','4room_sold','5room_sold','exec_sold','multigen_sold','studio_apartment_sold','1room_rental','2room_rental','3room_rental','other_room_rental','sec_sch_name']
housing2 = housing.drop(columns= cols_to_drop ,inplace=False)
housing2.head()

,id,tranc_yearmonth,flat_type,flat_model,resale_price,mid_storey,floor_area_sqft,hdb_age,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,planning_area,mall_within_500m,mall_within_1km,mall_within_2km,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,mrt_name,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,affiliation,train,indicator_to_drop,mature,top_10_pri_sch,cutoff_point_rank,percent_rental_flat,percent_4room_and_above_flat,distance_for_hawker_and_mrt,distance_for_mall_and_mrt
0,88471,2016-05,4 ROOM,Model A,680000.0,11,968.760,15,25,0,0,0,0,142,Kallang,0.0,0.0,7.0,1.0,3.0,13.0,84,60,Kallang,0,0,29.427395,1138.633422,78,1,1138.633422,0,1,0,1,0,27.0,0.0,1.0,5.108146e+04,3.611407e+05
1,122598,2012-07,5 ROOM,Improved,665000.0,8,1399.320,34,9,0,0,0,0,112,Bishan,0.0,1.0,3.0,0.0,1.0,7.0,80,77,Bishan,1,1,58.207761,415.607357,45,1,447.894399,0,1,0,1,0,20.0,0.0,1.0,5.784795e+05,7.834201e+05
2,170897,2013-07,EXECUTIVE,Apartment,838000.0,14,1550.016,24,16,0,0,0,0,90,Bukit Batok,0.0,0.0,4.0,0.0,0.0,1.0,84,95,Bukit Batok,1,0,214.747860,498.849039,39,0,180.074558,0,1,0,0,0,54.0,0.0,1.0,2.351060e+06,1.947446e+06
3,86070,2012-04,4 ROOM,Model A,550000.0,3,1108.692,29,11,1,0,0,0,75,Bishan,0.0,1.0,4.0,0.0,1.0,9.0,32,86,Bishan,1,1,43.396521,389.515528,20,1,389.515528,1,1,0,1,1,6.0,0.0,1.0,6.590066e+05,8.622398e+05
4,153632,2017-12,4 ROOM,Simplified,298000.0,2,893.412,34,4,0,0,0,0,48,Yishun,0.0,1.0,2.0,0.0,0.0,1.0,45,0,Khatib,0,0,129.422752,401.200584,74,0,312.025435,0,1,0,0,0,40.0,0.0,1.0,6.350707e+05,3.009164e+05


In [7]:
housing2.shape

(166542, 41)

In [8]:



#def calc_vif(X):

    # Calculating VIF
    #vif = pd.DataFrame()
    #vif["variables"] = X.columns
    #vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    #return(vif)

In [9]:
#X = housing2[housing2['train']==1].drop(columns=['flat_type','flat_model','planning_area','mrt_name'])

#calc_vif(X)

In [10]:
# Mutual information on Numerical Input
#from sklearn.feature_selection import mutual_info_regression

#y = housing2[housing2['train']==1]['resale_price']


#mi_scores = mutual_info_regression(X,y)
#mi_scores = pd.Series(mi_scores, name="MI Scores", index = X.columns)
#mi_scores = mi_scores.sort_values(ascending=False)

#mi_scores

### Creating dummy variables for categories variables

####  flat_type, flat_model, full_flat_type, planning_area, mrt_name, bus_stop_name, pri_sch_name,region, tranc_year, tranc_month, tranc_yearmonth 

In [11]:
#dummify_list = ['flat_type','flat_model','full_flat_type','planning_area','mrt_name','bus_stop_name','pri_sch_name','region','tranc_year','tranc_month','tranc_yearmonth']
dummify_list = ['flat_type','flat_model','mrt_name','planning_area','tranc_yearmonth'] #'mrt_name'

for column in dummify_list:
    # Create dummy variables for each column and concatenate them to the housing2 DataFrame
    dummy_variables = pd.get_dummies(housing[column], prefix=column, drop_first=True)
    housing2 = pd.concat([housing2, dummy_variables], axis=1)

# Drop the original categorical columns from the housing2 DataFrame
housing2.drop(columns=dummify_list, inplace=True)

housing2.head()

,id,resale_price,mid_storey,floor_area_sqft,hdb_age,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,mall_within_500m,mall_within_1km,mall_within_2km,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,affiliation,train,indicator_to_drop,mature,top_10_pri_sch,cutoff_point_rank,percent_rental_flat,percent_4room_and_above_flat,distance_for_hawker_and_mrt,distance_for_mall_and_mrt,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION,flat_model_Adjoined flat,flat_model_Apartment,flat_model_DBSS,flat_model_Improved,flat_model_Improved-Maisonette,flat_model_Maisonette,flat_model_Model A,flat_model_Model A-Maisonette,...,tranc_yearmonth_2017-03,tranc_yearmonth_2017-04,tranc_yearmonth_2017-05,tranc_yearmonth_2017-06,tranc_yearmonth_2017-07,tranc_yearmonth_2017-08,tranc_yearmonth_2017-09,tranc_yearmonth_2017-10,tranc_yearmonth_2017-11,tranc_yearmonth_2017-12,tranc_yearmonth_2018-01,tranc_yearmonth_2018-02,tranc_yearmonth_2018-03,tranc_yearmonth_2018-04,tranc_yearmonth_2018-05,tranc_yearmonth_2018-06,tranc_yearmonth_2018-07,tranc_yearmonth_2018-08,tranc_yearmonth_2018-09,tranc_yearmonth_2018-10,tranc_yearmonth_2018-11,tranc_yearmonth_2018-12,tranc_yearmonth_2019-01,tranc_yearmonth_2019-02,tranc_yearmonth_2019-03,tranc_yearmonth_2019-04,tranc_yearmonth_2019-05,tranc_yearmonth_2019-06,tranc_yearmonth_2019-07,tranc_yearmonth_2019-08,tranc_yearmonth_2019-09,tranc_yearmonth_2019-10,tranc_yearmonth_2019-11,tranc_yearmonth_2019-12,tranc_yearmonth_2020-01,tranc_yearmonth_2020-02,tranc_yearmonth_2020-03,tranc_yearmonth_2020-04,tranc_yearmonth_2020-05,tranc_yearmonth_2020-06,tranc_yearmonth_2020-07,tranc_yearmonth_2020-08,tranc_yearmonth_2020-09,tranc_yearmonth_2020-10,tranc_yearmonth_2020-11,tranc_yearmonth_2020-12,tranc_yearmonth_2021-01,tranc_yearmonth_2021-02,tranc_yearmonth_2021-03,tranc_yearmonth_2021-04
0,88471,680000.0,11,968.760,15,25,0,0,0,0,142,0.0,0.0,7.0,1.0,3.0,13.0,84,60,0,0,29.427395,1138.633422,78,1,1138.633422,0,1,0,1,0,27.0,0.0,1.0,5.108146e+04,3.611407e+05,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,122598,665000.0,8,1399.320,34,9,0,0,0,0,112,0.0,1.0,3.0,0.0,1.0,7.0,80,77,1,1,58.207761,415.607357,45,1,447.894399,0,1,0,1,0,20.0,0.0,1.0,5.784795e+05,7.834201e+05,0,0,0,1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,170897,838000.0,14,1550.016,24,16,0,0,0,0,90,0.0,0.0,4.0,0.0,0.0,1.0,84,95,1,0,214.747860,498.849039,39,0,180.074558,0,1,0,0,0,54.0,0.0,1.0,2.351060e+06,1.947446e+06,0,0,0,0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,86070,550000.0,3,1108.692,29,11,1,0,0,0,75,0.0,1.0,4.0,0.0,1.0,9.0,32,86,1,1,43.396521,389.515528,20,1,389.515528,1,1,0,1,1,6.0,0.0,1.0,6.590066e+05,8.622398e+05,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,153632,298000.0,2,893.412,34,4,0,0,0,0,48,0.0,1.0,2.0,0.0,0.0,1.0,45,0,0,0,129.422752,401.200584,74,0,312.025435,0,1,0,0,0,40.0,0.0,1.0,6.350707e+05,3.009164e+05,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
#Save to csv file
# Drop resale price outliner as it only contribute to 3% - from 02_EDA finding
#train = housing2[((housing2['train']==1) & (housing2['resale_price'] < 781888))]

train = housing2[housing2['train']==1]
test = housing2[housing2['train']==0]

train.to_csv('../data/regression_train.csv', index=False)

test.to_csv('../data/regression_test.csv', index=False)

In [13]:
housing2[housing2['train']==0]

,id,resale_price,mid_storey,floor_area_sqft,hdb_age,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,mall_within_500m,mall_within_1km,mall_within_2km,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,affiliation,train,indicator_to_drop,mature,top_10_pri_sch,cutoff_point_rank,percent_rental_flat,percent_4room_and_above_flat,distance_for_hawker_and_mrt,distance_for_mall_and_mrt,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION,flat_model_Adjoined flat,flat_model_Apartment,flat_model_DBSS,flat_model_Improved,flat_model_Improved-Maisonette,flat_model_Maisonette,flat_model_Model A,flat_model_Model A-Maisonette,...,tranc_yearmonth_2017-03,tranc_yearmonth_2017-04,tranc_yearmonth_2017-05,tranc_yearmonth_2017-06,tranc_yearmonth_2017-07,tranc_yearmonth_2017-08,tranc_yearmonth_2017-09,tranc_yearmonth_2017-10,tranc_yearmonth_2017-11,tranc_yearmonth_2017-12,tranc_yearmonth_2018-01,tranc_yearmonth_2018-02,tranc_yearmonth_2018-03,tranc_yearmonth_2018-04,tranc_yearmonth_2018-05,tranc_yearmonth_2018-06,tranc_yearmonth_2018-07,tranc_yearmonth_2018-08,tranc_yearmonth_2018-09,tranc_yearmonth_2018-10,tranc_yearmonth_2018-11,tranc_yearmonth_2018-12,tranc_yearmonth_2019-01,tranc_yearmonth_2019-02,tranc_yearmonth_2019-03,tranc_yearmonth_2019-04,tranc_yearmonth_2019-05,tranc_yearmonth_2019-06,tranc_yearmonth_2019-07,tranc_yearmonth_2019-08,tranc_yearmonth_2019-09,tranc_yearmonth_2019-10,tranc_yearmonth_2019-11,tranc_yearmonth_2019-12,tranc_yearmonth_2020-01,tranc_yearmonth_2020-02,tranc_yearmonth_2020-03,tranc_yearmonth_2020-04,tranc_yearmonth_2020-05,tranc_yearmonth_2020-06,tranc_yearmonth_2020-07,tranc_yearmonth_2020-08,tranc_yearmonth_2020-09,tranc_yearmonth_2020-10,tranc_yearmonth_2020-11,tranc_yearmonth_2020-12,tranc_yearmonth_2021-01,tranc_yearmonth_2021-02,tranc_yearmonth_2021-03,tranc_yearmonth_2021-04
149805,114982,NaN,8,904.176,34,12,1,0,0,0,132,0.0,2.0,4.0,0.0,1.0,1.0,56,123,0,0,75.683952,426.467910,92,0,156.322353,0,0,0,0,0,33.0,0.000000,1.000000,4.721318e+05,6.024975e+05,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149806,95653,NaN,5,1205.568,13,14,0,0,0,0,53,0.0,1.0,3.0,0.0,0.0,0.0,72,94,0,0,88.993058,439.756851,45,0,739.371688,0,0,0,0,0,47.0,0.000000,1.000000,3.596914e+05,9.050774e+04,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149807,40303,NaN,8,731.952,41,12,0,0,0,0,218,0.0,2.0,3.0,1.0,3.0,11.0,50,100,1,0,86.303575,355.882207,36,0,305.071191,0,0,0,1,0,9.0,0.013761,0.110092,1.057212e+05,5.672131e+05,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149808,109506,NaN,2,1044.108,42,14,0,0,0,0,104,0.0,0.0,3.0,0.0,1.0,2.0,52,112,0,0,108.459039,929.744711,54,0,433.454591,0,0,0,0,0,54.0,0.000000,1.000000,5.607157e+05,1.421907e+06,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
149809,100149,NaN,17,1108.692,36,25,0,0,0,0,144,0.0,0.0,5.0,1.0,2.0,2.0,60,87,0,0,113.645431,309.926934,40,0,217.295361,0,0,0,0,0,28.0,0.000000,1.000000,4.323319e+05,1.058086e+06,0,0,1,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..